In [1]:
from behavior.behIO import behIO
from behavior.plotScale import plotScale
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle
#%matplotlib inline

In [2]:
colors = ['Green', 'Blue', 'Red', 'Grey']

stylish = {'axes.axisbelow': False,
 'axes.edgecolor': '1',
 'axes.facecolor': 'white',
 'axes.grid': False,
 'axes.labelcolor': '.15',
 'axes.spines.bottom': False,
 'axes.spines.left': False,
 'axes.spines.right': False,
 'axes.spines.top': False,
 'figure.facecolor': 'white',
 'font.family': ['sans-serif'],
 'font.sans-serif': ['Arial',
  'DejaVu Sans',
  'Liberation Sans',
  'Bitstream Vera Sans',
  'sans-serif'],
 'font.size': 14,
 'grid.color': '.8',
 'grid.linestyle': '-',
 'image.cmap': 'rocket',
 'lines.solid_capstyle': 'round',
 'patch.edgecolor': 'w',
 'patch.force_edgecolor': True,
 'text.color': '.15',
 'xtick.bottom': False,
 'xtick.color': '.15',
 'xtick.direction': 'out',
 'xtick.top': False,
 'ytick.color': '.15',
 'ytick.direction': 'out',
 'ytick.left': False,
 'ytick.right': False}

In [3]:
mouse = 'LR8'
#dates = ['180713', '180719']
dates = ['180731']
runs = [1, 2, 3]

In [4]:
for date in dates:
    # Set up an experiment
    mus = behIO('LR8', date, runs)
    mus.setDir('/Volumes/User Folders/Trent/MixedValenceBehavior')
    #mus.setDir('/Users/Amelia/Documents/hakan/behavior')

    # Load ML and NIDAQ data
    mus.loadBHV()
    mus.loadRuns('nidaq')

    for run in mus.runs:
        # Get trial info, onsets and offsets for each event
        mus.setRun(run= run) # Set the target run
        mus.getTrials()
        mus.getNidaq(clean= True)
        mus.findOnsets()
        mus.getOnsets()
        Fs = mus.getFs()

        preITI = 2.0 #sec
        postITI = 6.0 #sec
        time = np.linspace(-preITI, postITI, (postITI+preITI) * mus.nidaq.Fs)

        # Find the visual stim onsets and so we will organize the data around the vis stim onsets
        # Trial bins
        mus.onsetMatrix['trialnum'] = pd.Series(np.zeros(len(mus.onsetMatrix['timestamps']), dtype= int), dtype= int)
        mus.onsetMatrix['time'] = pd.Series(np.zeros(len(mus.onsetMatrix['timestamps']), dtype= int), dtype= int)
        mus.onsetMatrix['condition'] = pd.Series(np.zeros(len(mus.onsetMatrix['timestamps']), dtype= int), dtype= int)
        mus.onsetMatrix['trialerrors'] = pd.Series(np.zeros(len(mus.onsetMatrix['timestamps']), dtype= int), dtype= int)

        visOnsets = mus.pulseOnsets['visual_stimulus']['onsetInd'].values
        bins = np.reshape(np.concatenate([visOnsets - preITI*Fs, visOnsets + postITI*Fs]), (2, visOnsets.size)).transpose().astype(int)

        for i in range(0, len(mus.bhv['trialnum'])):
            begin = bins[i,:][0]
            end = bins[i,:][1]

            # add the trial number
            mus.onsetMatrix['trialnum'].iloc[begin : end] = mus.bhv['trialnum'].loc[i]

            # add the offset time
            mus.onsetMatrix['time'].iloc[begin : end] = time

            # add the conditions
            mus.onsetMatrix['condition'].iloc[begin : end] = mus.bhv['conditions'].loc[i]

            # add the trial errors
            mus.onsetMatrix['trialerrors'].iloc[begin : end] = mus.bhv['trialerrors'].loc[i]

        onsets = mus.onsetMatrix[mus.onsetMatrix['trialnum'] > 0].reset_index(drop= True)

        # Do the same to offsets
        # Trial bins
        mus.offsetMatrix['trialnum'] = pd.Series(np.zeros(len(mus.offsetMatrix['timestamps']), dtype= int), dtype= int)
        mus.offsetMatrix['time'] = pd.Series(np.zeros(len(mus.offsetMatrix['timestamps']), dtype= int), dtype= int)
        mus.offsetMatrix['condition'] = pd.Series(np.zeros(len(mus.offsetMatrix['timestamps']), dtype= int), dtype= int)
        mus.offsetMatrix['trialerrors'] = pd.Series(np.zeros(len(mus.offsetMatrix['timestamps']), dtype= int), dtype= int)

        visOnsets = mus.pulseOnsets['visual_stimulus']['onsetInd'].values
        bins = np.reshape(np.concatenate([visOnsets - preITI*Fs, visOnsets + postITI*Fs]), (2, visOnsets.size)).transpose().astype(int)

        for i in range(0, len(mus.bhv['trialnum'])):
            begin = bins[i,:][0]
            end = bins[i,:][1]

            # add the trial number
            mus.offsetMatrix['trialnum'].iloc[begin : end] = mus.bhv['trialnum'].loc[i]

            # add the offset time
            mus.offsetMatrix['time'].iloc[begin : end] = time

            # add the conditions
            mus.offsetMatrix['condition'].iloc[begin : end] = mus.bhv['conditions'].loc[i]

            # add the conditions
            mus.offsetMatrix['trialerrors'].iloc[begin : end] = mus.bhv['trialerrors'].loc[i]

        offsets = mus.offsetMatrix[mus.offsetMatrix['trialnum'] > 0].reset_index(drop= True)

        firstShock = False
        firstLicks = True
        conditions = set(onsets.condition)

        # First organize and sort the data according to trial condition and trial error
        #events = onsets.pivot_table(index= ['condition', 'trialnum'], columns= 'time')
        events = onsets.pivot_table(index= ['condition', 'trialnum', 'trialerrors'], columns= 'time').sort_index(level= [0, 2])
        licks = np.array(events['licking'].values * time)
        shocks = np.array(events['shock'].values * time)
        ensure = np.array(events['ensure'].values * time)

        #eventsOff = offsets.pivot_table(index= ['condition', 'trialnum'], columns= 'time')
        eventsOff = offsets.pivot_table(index= ['condition', 'trialnum'], columns= 'time')
        shocksOff = np.array(eventsOff['shock'].values * time)
        ensureOff = np.array(eventsOff['ensure'].values * time)

        sns.set_style(stylish)
        fig, ax = plt.subplots(figsize= (7.5, 10))

        for i in range(0, licks.shape[0]):
            # Plot reward
            if len(ensure[i,:][ensure[i,:] > 0]) > 0:
                #ensureLine = ax.vlines(ensure[i,:][ensure[i,:] != 0][0], i-0.35, i+0.35, linewidth= 1, color='g', alpha= 0.3, label= 'Ensure')
                #ensureLine = ax.hlines(i, ensure[i,:][ensure[i,:] != 0][0], ensureOff[i,:][ensureOff[i,:] != 0][0], linewidth= 5, color='g', alpha= 0.3, label= 'Ensure')
                if ~len(ensureOff[i,:][ensureOff[i,:] != 0]):
                    ensureLine = ax.hlines(i, ensure[i,:][ensure[i,:] != 0][0], ensure[i,:][ensure[i,:] != 0][0]+0.5, linewidth= 5, color='g', alpha= 0.3, label= 'Ensure')
                else:
                    ensureLine = ax.hlines(i, ensure[i,:][ensure[i,:] != 0][0], ensureOff[i,:][ensureOff[i,:] != 0][0], linewidth= 5, color='g', alpha= 0.3, label= 'Ensure')
            # Plot shocks
            if firstShock:
                shockLines = ax.vlines(shocks[i,:][shocks[i,:] != 0][0], i-0.35, i+0.35, linewidth= 1, color='r', alpha= 0.3, label= 'Shock')
            else:
                if len(shocks[i,:][shocks[i,:] != 0]) > 0:
                    #shockLines = ax.vlines(shocks[i,:][shocks[i,:] != 0], i-0.35, i+0.35, linewidth= 1, color='r', alpha= 0.4)
                    shockLines = ax.hlines(i, shocks[i,:][shocks[i,:] != 0][0], shocks[i,:][shocks[i,:] != 0][-1], linewidth= 5, color='r', alpha= 0.3, label= 'Shock')
            # Plot licks
            lickLines = ax.vlines(licks[i,:][licks[i,:] != 0], i-0.35, i+0.35, linewidth= 1, color='k', label= 'Lick')
            # Mark the first lick
            if firstLicks:
                firstLick = licks[i,:][licks[i,:] != 0]
                if len(firstLick[firstLick > 0]) > 0:
                    firstLickLine = ax.vlines(firstLick[firstLick > 0][0], i-0.35, i+0.35, linewidth= 2, color='b', alpha= 0.5, label= 'First lick')

        ax.invert_yaxis()
        ax.set_xlim([-preITI - 2, postITI])
        ax.set_yticks([])
        ax.set_xticks(np.arange(-preITI, postITI + 0.1, 2))
        ax.set_xlabel('time (sec)')


        width = 0.2
        count = 0
        for con in conditions:
            trials = events.loc[con].shape[0]

            # A lot of effort to set the label
            centerx = -preITI - width*4
            centery = count + trials/2
            if mus.bhv.codes[con].split("_")[0] == 'CSmix':
                ylabel = mus.bhv.codes[con].split("_")[0]
            elif mus.bhv.codes[con].split("_")[0] == 'Blank':
                ylabel = mus.bhv.codes[con].split("_")[0]
            else:
                ylabel = mus.bhv.codes[con].split("_")[1]
            ax.text(centerx, centery, ylabel, rotation= 'vertical', va= 'center')

            if count > 0:
                ax.hlines(count-0.5, -preITI - width*2, postITI, linewidth= 0.25)

            rect = Rectangle((-preITI - width*2, count), width, trials-1, linewidth= 0, facecolor= colors[con-1])
            count = count + trials
            ax.add_patch(rect)

        cueBox = Rectangle((0, -1), 2, count+1, alpha= 0.2, facecolor= 'Grey' , label= 'Cue')
        ax.add_patch(cueBox)

        ax.set_title(mus.mouse + ', ' + mus.date + ', Run_' + str(mus.run))

        handles, labels = ax.get_legend_handles_labels()
        legend = ax.legend(handles=[lickLines, firstLickLine, ensureLine, shockLines, cueBox], frameon=False, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

        plt.tight_layout()

        # Save the figure
        fig.savefig(mus.savePath('licks', 'pdf'), bbox_extra_artists=(legend,), bbox_inches='tight')
        print(mus.mouse, mus.date, str(mus.run), 'Figure saved.')


Base directory is set to /Volumes/User Folders/Trent/MixedValenceBehavior
All runs will be loaded.
Finished loading ML data.

All runs will be loaded.
Finished loading run(s).



/Users/Amelia/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/Amelia/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


IndexError: index 117 is out of bounds for axis 0 with size 117

In [ ]:
LR8 = behIO('LR8', '180717', [1,2,3])
#LR8.setDir('/Volumes/User Folders/Trent/MixedValenceBehavior')
LR8.setDir('/Users/Amelia/Documents/hakan/behavior')

# Load ML and NIDAQ data
LR8.loadBHV()
LR8.loadRuns('nidaq')


In [ ]:
# Set up an experiment
dates = ['180720', '180719', '180717']
runs = [1,2,3]

# Set the trial basics
preITI = 4.0 #sec
postITI = 8.0 #sec

In [ ]:
bigMatrix = {}
for date in dates:
    LR8 = behIO('LR8', date, runs)
    #LR8.setDir('/Volumes/User Folders/Trent/MixedValenceBehavior')
    LR8.setDir('/Users/Amelia/Documents/hakan/behavior')
    
    # Load ML and NIDAQ data
    LR8.loadBHV()
    LR8.loadRuns('nidaq')
        
    bigRun = {}
    for run in runs:
        # Get trial info, onsets and offsets for each event
        LR8.setRun(run= run) # Set the target run
        LR8.getTrials()
        LR8.getNidaq()
        LR8.findOnsets()
        LR8.getOnsets()
        Fs = LR8.getFs()
 
        bigRun[run] = LR8.onsetMatrix
    
    bigMatrix[date] = bigRun